In [1]:
import os
SUBMIT_ARGS = "--packages databricks:spark-deep-learning:1.0.0-spark2.3-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DL with Spark Deep Cognition").getOrCreate()
sc = spark.sparkContext

In [3]:
sc = spark.sparkContext
sc

Out[ 3 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [4]:
# load image

In [5]:
display(dbutils.fs.ls("dbfs:/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/cle/,cle/,0
dbfs:/FileStore/tables/cre/,cre/,0
dbfs:/FileStore/tables/ole/,ole/,0
dbfs:/FileStore/tables/ore/,ore/,0


In [6]:
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
# Read images using Spark ... as a DataFrame.
# Each image is stored as a row in the imageSchema format.
image_cle = ImageSchema.readImages("dbfs:/FileStore/tables/cle/").withColumn("label", lit(0))
image_cre = ImageSchema.readImages("dbfs:/FileStore/tables/cre/").withColumn("label", lit(1))
image_ole = ImageSchema.readImages("dbfs:/FileStore/tables/ole/").withColumn("label", lit(2))
image_ore = ImageSchema.readImages("dbfs:/FileStore/tables/ore/").withColumn("label", lit(3))

In [7]:
image_cle.show(),image_cre.show(),image_ole.show(),image_ore.show()

+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
[dbfs:/FileStore/...| 0|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
[dbfs:/FileStore/...| 1|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
[dbfs:/FileStore/...| 2|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
 image|label|
+--------------------+-----+
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
[dbfs:/FileStore/...| 3|
+--------------------+-----+
only showing top 20 rows

 Out[ 17 ]: (None, None, None, None)

In [8]:
type(image_cle)

Out[ 204 ]: pyspark.sql.dataframe.DataFrame

In [9]:
# Create training & test DataFrames for transfer learning - this piece of code is longer than transfer learning itself below!
re_df = image_ore.unionAll(image_cre)
le_df= image_ole.unionAll(image_cle)

re_train, re_test = re_df.randomSplit([0.6, 0.4])  # use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
le_train, le_test= le_df.randomSplit([0.6, 0.4])     # use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)

train_df = re_train.unionAll(le_train)
test_df = re_test.unionAll(le_test)

# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [10]:
#re_df.show(),le_df.show()
train_df.show(),test_df.show()

In [11]:
train_df.count(),test_df.count()

Out[ 20 ]: (2938, 1908)

In [12]:
#image_ore.count(),image_cle.count(),re_df.count(),re_train.count(),re_test.count()

In [13]:
# Transfer Learning technique
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

Using TensorFlow backend.

In [15]:
p_model = p.fit(train_df)

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

Test set accuracy = 0.868759650026

In [17]:
tested_df.show()

+--------------------+-----+--------------------+--------------------+--------------------+----------+
 image|label| features| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
[dbfs:/FileStore/...| 3|[0.0,0.5171633362...|[-1.3014686635494...|[0.03035875350513...| 3.0|
[dbfs:/FileStore/...| 3|[0.11813035607337...|[-1.4375764635558...|[0.02990164701562...| 3.0|
[dbfs:/FileStore/...| 3|[0.0,0.0,0.0,0.0,...|[-2.1369329298498...|[0.01377528072229...| 3.0|
[dbfs:/FileStore/...| 3|[0.0,0.0,0.0,0.0,...|[-0.6209392170818...|[0.08189854282282...| 3.0|
[dbfs:/FileStore/...| 3|[0.22917024791240...|[-1.0870279007641...|[0.05513727706700...| 3.0|
[dbfs:/FileStore/...| 1|[0.30938124656677...|[0.19760757755752...|[0.01801682913611...| 1.0|
[dbfs:/FileStore/...| 1|[0.0,0.0,0.0,0.0,...|[1.05523505669528...|[0.41538139452345...| 1.0|
[dbfs:/FileStore/...| 1|[0.0,0.0,0.593968...|[-0.9045189991381...|[0.06888506750738...| 3.0|
[dbfs:/FileStore/...| 1|[0.0,1.1365066766...|[0.07515362995718...|[0.01401873802597...| 1.0|
[dbfs:/FileStore/...| 1|[0.0,0.0,0.0,0.0,...|[0.97327098321437...|[0.41826573202145...| 0.0|
[dbfs:/FileStore/...| 2|[0.56483000516891...|[0.25705866046310...|[0.14871442033336...| 2.0|
[dbfs:/FileStore/...| 2|[0.0,0.0,0.0,0.70...|[-1.1019050710568...|[0.03654474426324...| 2.0|
[dbfs:/FileStore/...| 2|[0.0,0.0,0.0,0.03...|[-1.7860344311514...|[0.00768770854498...| 2.0|
[dbfs:/FileStore/...| 2|[1.51130294799804...|[-0.7656064415679...|[0.06882899406327...| 2.0|
[dbfs:/FileStore/...| 2|[1.19859373569488...|[-1.0550999087373...|[0.05458817439646...| 2.0|
[dbfs:/FileStore/...| 0|[1.71732568740844...|[3.06820553687133...|[0.89605402270346...| 0.0|
[dbfs:/FileStore/...| 0|[0.0,0.0,0.0,0.0,...|[3.21657953549990...|[0.91025173971549...| 0.0|
[dbfs:/FileStore/...| 0|[0.0,0.9574085474...|[1.40690891869710...|[0.55646458611677...| 0.0|
[dbfs:/FileStore/...| 0|[0.0,0.0,0.0,0.31...|[2.69620623638942...|[0.80995395150815...| 0.0|
[dbfs:/FileStore/...| 3|[0.68790531158447...|[-2.6513035127232...|[0.00275027656502...| 3.0|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [18]:
from sparkdl import DeepImagePredictor

In [19]:
image_df = ImageSchema.readImages("/FileStore/tables/salma3.jpg")

In [20]:
predictions_df = p_model.transform(image_df)

In [21]:
predictions_df.show()

+--------------------+--------------------+--------------------+--------------------+----------+
 image| features| rawPrediction| probability|prediction|
+--------------------+--------------------+--------------------+--------------------+----------+
[dbfs:/FileStore/...|[0.0,0.4746651947...|[0.82698075160750...|[0.30487671925010...| 1.0|
+--------------------+--------------------+--------------------+--------------------+----------+

In [22]:
predictions_df.select("prediction").show(truncate=False,n=3)

+----------+
prediction|
+----------+
1.0 |
+----------+

In [23]:
x=predictions_df

In [24]:
dddd = x.toPandas()
dddd

Out[ 181 ]: 
 image \
0 (dbfs:/FileStore/tables/salma3.jpg, 82, 102, 4... 

 features \
0 [0.0, 0.47466519474983215, 0.46041661500930786... 

 rawPrediction \
0 [0.8269807516075073, 1.3504697718922756, -0.39... 

 probability prediction 
0 [0.3048767192501069, 0.5146034046841377, 0.090... 1.0

In [25]:
y=dddd['prediction'][0]

In [26]:
y = int(y)
type(y)

Out[ 183 ]: int

In [27]:
def class_predict(y):
  if y==0:
    return('close left eye')
  elif y==1:
    return('close right eye')
  elif y==2:
    return('open left eye')
  elif y==3:
     return('open right eye')
  else:
    return ('error')

In [28]:
class_predict(y)

Out[ 185 ]: 'close right eye'